In [1]:
import xml.etree.ElementTree as ET
import requests

In [2]:
# nome dos ficheiros a criar

f = open('flickr.csv', 'w', encoding="utf-8")
g = open('smallboxes.csv', 'w', encoding="utf-8")

In [3]:
#create the cursor
#cur = conn.cursor
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.search&api_key=d827ec49d172cace3be06ec605a30cb0&per_page=250'

In [4]:
##Coordenadas do rectangulo que abrange a area de estudo, graus decimais WGS84 
bboxlist = [[-9.229836,38.690188,-9.086333,38.803988]]
#initialize the smallbox list
smallboxes = []

In [5]:
#current box check for the total number of photos
def check_current_box(curbox):
    stringBox = str(curbox).strip("[]")
    args = { 'min_taken_date' : '2016-01-01', # data de inicio de recolha dos dados
             'max_taken_date' : '2021-09-29', # data de fim de recolha de dados
             'bbox' : stringBox,
             'extras' : 'geo, url_m, tags, views, date_taken, date_upload, media, owner_name'}
    box = requests.get(url, params = args)
    #need to parse element 'photos' in order to get
    #this will need to be debugged
    checkBox = ET.fromstring(box.content)
    total = int(checkBox[0].attrib['total'])
    nopages = int(checkBox[0].attrib['pages'])
    return {"total": total, "pages": nopages}

In [6]:
#function to get small enough boxes
def get_small_boxes(boundingbox):
    while len(boundingbox) > 0:
        for e in boundingbox:
            #get the current box
            currentbox = e
            #insert check_current_box here
            check = check_current_box(currentbox)
            nophotos, nopages = check['total'], check['pages']
            if nophotos >= 501:
                print ("[*] Too many photos in the current box %s, %s, %s" % (currentbox, nophotos, nopages) )
                #find the height of the current box
                h = (abs(e[3] - e[1]) * .5)
                #find the width of the current box
                w = (abs(e[0] - e[2]) * .5)
                new_lat = e[1] + h
                new_lon = e[0] + w
                nb1 = [e[0], new_lat, new_lon, e[3]]
                bboxlist.append(nb1)
                nb2 = [new_lon, new_lat, e[2], e[3]]
                bboxlist.append(nb2)
                nb3 = [e[0], e[1], new_lon, new_lat]
                bboxlist.append(nb3)
                nb4 = [new_lon, e[1], e[2], new_lat]
                bboxlist.append(nb4)
                index = boundingbox.index(e)
                bboxlist.pop(index)
            else:
                print ("[*] This box is small enough %s, %s, %s" % (currentbox, nophotos, nopages) )
                minilist = [e, nopages]
                smallboxes.append(minilist)
                get_data(e, nopages)
                index = boundingbox.index(e)
                bboxlist.pop(index)

def get_data(box, pages):
    curpage = 1
    while curpage <= pages:
        stringBox = str(box).strip("[]")
        args = {'bbox' : stringBox,
                    'min_taken_date' : '2016-01-01', # data de inicio de recolha dos dados
                    'max_taken_date' : '2021-09-29', # data de fim de recolha de dados
                    'extras' : 'geo, url_m, tags, views, date_taken, date_upload, media'}
        result = requests.get(url, params = args)
        data = ET.fromstring(result.content)
        curpage += 1
        for photo in data[0]:
            try:
                pid = str(photo.attrib['id'])
            except:
                pid = "PID problem"
            try:
                owner = str(photo.attrib['owner'])
            except:
                owner  = "owner problem"
            try:
                secret = str(photo.attrib['secret'])
            except:
                secret = "secret problem"
            try:
                server = str(photo.attrib['server'])
            except:
                server = "sever problem"
            try:
                farm = str(photo.attrib['farm'])
            except:
                farm = "farm problem"
            try:
                latitude = str(photo.attrib['latitude'])
            except:
                latitude = "latitude problem"
            try:    
                longitude = str(photo.attrib['longitude'])
            except:
                longitude = "longitude problem"
            try:
                woeid = str(photo.attrib['woeid'])
            except:
                woeid = "woied problem"
            try:
                place_id = str(photo.attrib['place_id'])
            except:
                place_id = "place_id problem"
            try:
                url_m =  str(photo.attrib['url_m'])
            except:
                url_m = "url_m problem"
            try:
                tags = str(photo.attrib['tags'])
            except:
                tags = "tags problem"
            try:
                views = str(photo.attrib['views'])
            except:
                views = "views problem"
            try:
                date_t = str(photo.attrib['datetaken'])
            except:
                date_t = "date_t problem"
            try:
                date_g = str(photo.attrib['datetakengranularity'])
            except:
                date_g = "date_g problem"        
            try:
                date_u = str(photo.attrib['dateupload'])
            except:
                date_u = "date_u problem"
            try:
                media = str(photo.attrib['media'])
            except:
                media = "media problem"
            try:
                title =str(photo.attrib['title'])
            except:
                title = "title problem"                
    
            line = "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n" %(   pid,            
                                                                               owner,
                                                                               secret,
                                                                               server,
                                                                               farm,
                                                                               latitude, 
                                                                               longitude,
                                                                               woeid,
                                                                               place_id,
                                                                               url_m,
                                                                               tags,
                                                                               views,
                                                                               date_t,
                                                                               date_g,
                                                                               date_u,
                                                                               media,
                                                                               title)
            
            f.write(line)
            print ("writing line")

get_small_boxes(bboxlist)


#this writes the small boxes list to a file.
for box in smallboxes:
    sbox = str(box)
    g.write(sbox + '"\n')

KeyboardInterrupt: 